# Using DBSCAN as clustering method

In [3]:
import numpy as np
import pandas as pd
import re
from sklearn import preprocessing
from scipy.sparse import csr_matrix
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from matplotlib import pyplot as plt

## Test

In [58]:
states = ["INITIAL","login","View_Items","home","logout","View_Items_quantity","Add_to_Cart","shoppingcart",
          "remove","deferorder","purchasecart","inventory","sellinventory","clearcart","cancelorder","$"]
sessions = {'HZKS0-WG8pZr0eCsZlBAP5Xm': ['login',
  'View_Items',
  'View_Items',
  'View_Items',
  'View_Items',
  'View_Items',
  'View_Items',
  'View_Items',
  'View_Items',
  'View_Items',
  'View_Items',
  'View_Items',
  'View_Items',
  'View_Items',
  'View_Items',
  'home',
  'logout'],
 '5tPgZbHdK2Zp+heFBs8HsMkx': ['login',
  'View_Items_quantity',
  'Add_to_Cart',
  'View_Items_quantity',
  'Add_to_Cart',
  'View_Items_quantity',
  'Add_to_Cart',
  'View_Items_quantity',
  'Add_to_Cart',
  'View_Items_quantity',
  'Add_to_Cart',
  'shoppingcart',
  'remove',
  'shoppingcart',
  'remove',
  'shoppingcart',
  'remove',
  'shoppingcart',
  'remove',
  'deferorder',
  'home',
  'logout'],
 'RU2oCVNdpWEM0-2x7I5OjPbZ': ['login',
  'View_Items_quantity',
  'Add_to_Cart',
  'View_Items_quantity',
  'Add_to_Cart',
  'purchasecart',
  'home',
  'logout'],
 'kG4g0E5mqwRYcsQOCfj+7wG7': ['login',
  'inventory',
  'inventory',
  'sellinventory',
  'sellinventory',
  'sellinventory',
  'home',
  'logout'],
 '8ocO6WP3QaFpBvkooS5INPwe': ['login',
  'inventory',
  'inventory',
  'sellinventory',
  'sellinventory',
  'sellinventory',
  'sellinventory',
  'sellinventory',
  'sellinventory',
  'home',
  'logout'],
 'WOTZQBwSCnI+DfDQ-2cS7Mgp': ['login',
  'inventory',
  'inventory',
  'sellinventory',
  'sellinventory',
  'sellinventory',
  'sellinventory',
  'home',
  'logout'],
 'e4bMe1HfiUlmvUMmPJU4y1B4': ['login',
  'inventory',
  'inventory',
  'sellinventory',
  'sellinventory',
  'sellinventory',
  'home',
  'logout'],
 'MEzDpcnm1MQ9GFGox7uP4Ep-': ['login',
  'View_Items',
  'View_Items',
  'View_Items',
  'View_Items',
  'View_Items',
  'View_Items',
  'View_Items',
  'View_Items',
  'View_Items',
  'View_Items',
  'View_Items',
  'View_Items',
  'View_Items',
  'View_Items',
  'home',
  'logout']}

### Makovchain & sparse matrix

In [59]:
def transition_matrix(sessions, states):
    markovchains = []
    for key, value in sessions.items():
        # labelEncoding
        le = preprocessing.LabelEncoder()
        le.fit(value)
        transformed_s = le.transform(value)
        
        #factorize
        factorize = pd.factorize(value)[0]
        
        # matrix
        n = 1 + max(factorize)  # number of states
        M = [[0] * n for _ in range(n)]

        for (i, j) in zip(factorize, factorize[1:]):
            M[i][j] += 1
        
        # now convert to probabilities:
        for row in M:
            s = sum(row)
            if s > 0:
                row[:] = [f / s for f in row]
                
        # print Matrix style
        #for row in M: print(' '.join('{0:.2f}'.format(x) for x in row))
        
        # unique array in the right order
        value = np.array(value)
        _, idx = np.unique(value, return_index=True)
        
        df = pd.DataFrame(data = M, index=value[np.sort(idx)],
                          columns=value[np.sort(idx)])

        df_1 = pd.DataFrame(index=states, columns=states, dtype='float64')

        #merge = df_1.merge(df, how='right').fillna(0).round(2).set_index(value[np.sort(idx)])
        merge = df_1.update(df, join='left')
        #merge = pd.merge(df_1, df, how='right').fillna(0).round(2).set_index(value[np.sort(idx)])
        
        
        merge = pd.concat([pd.concat([df_1, df], axis=1, sort= False)], axis=0).fillna(0).round(2).iloc[:, :-n] 
        #merge = merge.iloc[:, :-n]        
        
        
        # convert into Vector
        merge = np.array(merge.values.flatten().tolist())
        #print(len(merge))
        # resize so the vectors got the same length
        #size = 16*16
        #merge.resize(size)
        
        # 2-D array 
        markovchains.append(merge)
        
        
        # csr sparse matrix
        csr = csr_matrix(markovchains)
        #print(merge)
        #markovchains.append(merge)
        
    #print(len(merge))
    return csr

m = transition_matrix(sessions, states)
m

<8x256 sparse matrix of type '<class 'numpy.float64'>'
	with 47 stored elements in Compressed Sparse Row format>

In [62]:
X = m
clustering = DBSCAN(eps=2, min_samples=2).fit(X)
labels = clustering.labels_
print(labels)
print(np.unique(labels, return_counts=True))
print(clustering)

[ 0 -1 -1  0  0  0  0  0]
(array([-1,  0], dtype=int32), array([2, 6]))
DBSCAN(algorithm='auto', eps=2, leaf_size=30, metric='euclidean',
    metric_params=None, min_samples=2, n_jobs=None, p=None)


## DBSCAN
Testing DBSCAN with random data + plot